In [20]:
camera.stop()
robot.stop()

### Start the ZED2i Camera system

In [19]:
from ultralytics import YOLO
model = YOLO("yolo11l_half.engine")

import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
import ipywidgets.widgets as widgets
from IPython.display import display
import collections
import motors
from traitlets.config.configurable import HasTraits
from datetime import timedelta, datetime

robot = motors.MotorsYukon(mecanum=False)

TARGET_DISTANCE = 1000
SCREEN_WIDTH = 672
CENTER_X = SCREEN_WIDTH/2  

display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside)

class Camera(HasTraits):
    color_value = traitlets.Any()
    person_pos = traitlets.Any()
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()

        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

        self.depth_image = np.asanyarray(self.depth.get_data())

        self.tracked_bbox = None
        self.tracked_depth = 0
        self.new_target_time = None
        self.x_center_t = None
        self.y_center_t = None

    
    def _capture_frames(self):

        while(self.thread_runnning_flag==True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag=True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join() 
            self.zed.close()

    @traitlets.observe('color_value')
    def processFrame(self, change):
        
        

        frame = change['new']
        result = model(frame,verbose=False)[0]
    
        trim=[0,0,0,0]
    
        conf_threshold = .6
        
    
        chosen_box = None
        
        for i in range (len(result.boxes.cls)):
            if(result.boxes.cls[i] == 0):
                if (result.boxes.conf[i] > conf_threshold):
                    bbox = result.boxes.xyxy[i]
                    cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)   
    
                    if self.tracked_bbox is None:
                        chosen_box = bbox
                        break;
                    
                    if chosen_box is None:
                        chosen_box = bbox
                    else:
                        x_center_c = (chosen_box[0] + chosen_box[2]) / 2
                        y_center_c = (chosen_box[1] + chosen_box[3]) / 2
                        x_center = (bbox[0] + bbox[2]) / 2
                        y_center = (bbox[1] + bbox[3]) / 2
    
                        distance_new = math.sqrt((x_center - self.x_center_t)**2 + (y_center - self.y_center_t)**2)
                        distance_chosen = math.sqrt((x_center_c - self.x_center_t)**2 + (y_center_c - self.y_center_t)**2)
    
                        if distance_new < distance_chosen:
                            chosen_box = bbox
                            
    
        if chosen_box is None:
            return;
        
        trim = [int(chosen_box[1]), int(chosen_box[3]), int(chosen_box[0]), int(chosen_box[2])]
        trim[0] = int(chosen_box[1])
        trim[1] = int(chosen_box[3])
        trim[2] = int(chosen_box[0])
        trim[3] = int(chosen_box[2])
        trimmed_depth_chosen_image = camera.depth_image[trim[0]:trim[1], trim[2]:trim[3]]


        trimmed_depth_chosen_image_cleaned = np.nan_to_num(trimmed_depth_chosen_image, nan=0.0).astype(np.float32)
        
        trimmed_depth_chosen_image_cleaned[trimmed_depth_chosen_image_cleaned == 0] = np.nan

        trimmed_depth_chosen_image_cleaned[trimmed_depth_chosen_image_cleaned<200]=200
        trimmed_depth_chosen_image_cleaned[trimmed_depth_chosen_image_cleaned>3000]=3000
        chosen_depth = np.nanmin(trimmed_depth_chosen_image_cleaned)
        is_tracked_box = False
        if self.tracked_bbox != None:
            if chosen_depth is not None or not np.isnan(chosen_depth):
                if abs(self.tracked_depth - int(chosen_depth))  > 700:
                    print(f"NEW TARGET: {abs(self.tracked_depth - int(chosen_depth))} \r")
                    if (self.new_target_time != None and self.new_target_time < datetime.now()):
                        self.new_target_time = None
                        self.tracked_bbox = chosen_box
                        is_tracked_box = True
                        print(f"MOVING TO NEW TARGET: {abs(self.tracked_depth - int(chosen_depth))} \r")
                    elif (self.new_target_time == None):
                        self.new_target_time = datetime.now() + timedelta(seconds=2)
                        # pass
                    # elif (self.new_target_time != None):
                    #     pass
                else:
                    self.tracked_bbox = chosen_box
                    is_tracked_box = True
            else:
                self.tracked_bbox = chosen_box
                is_tracked_box = True
        else:
            self.tracked_bbox = chosen_box
            is_tracked_box = True
        if is_tracked_box:
            self.new_target_time = None

        # if self.timecheck != None:
        #     print(datetime.now() - self.timecheck)
        # self.timecheck = datetime.now()
        
        self.x_center_t = (self.tracked_bbox[0] + self.tracked_bbox[2]) / 2
        self.y_center_t = (self.tracked_bbox[1] + self.tracked_bbox[3]) / 2
        
        radius = 10
        color = (0, 255, 0)
        thickness = 2
        cv2.circle(frame, (int(self.x_center_t), int(self.y_center_t)), radius, color, thickness)
        cv2.circle(frame, (int(672/2), int(168)), radius, color, thickness)
        
        scale = 0.1 
    
        resized_image = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        display_color.value = bgr8_to_jpeg(resized_image)
        
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        
        depth_colormap[:trim[0], :] = 0
        depth_colormap[trim[1]:, :] = 0
        depth_colormap[:, :trim[2]] = 0
        depth_colormap[:, trim[3]:] = 0
    
    
        resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
        trimmed_depth_image = camera.depth_image[trim[0]:trim[1], trim[2]:trim[3]]
    
        
        trimmed_depth_image_cleaned = np.nan_to_num(trimmed_depth_image, nan=0.0).astype(np.float32)
        
        trimmed_depth_image_cleaned[trimmed_depth_image_cleaned == 0] = np.nan
    
        trimmed_depth_image_cleaned[trimmed_depth_image_cleaned<200]=200
        trimmed_depth_image_cleaned[trimmed_depth_image_cleaned>3000]=3000

        try:
            average_depth = np.nanmin(trimmed_depth_image_cleaned)
            if is_tracked_box:
                self.tracked_depth = average_depth
        except:
            average_depth = np.nanmean(trimmed_depth_image_cleaned)
            if is_tracked_box:
                self.tracked_depth = average_depth
            print("crashed")
        self.person_pos = (average_depth, self.x_center_t, is_tracked_box)

    @traitlets.observe('person_pos')
    def handleMotion(self, change):
        depth, x_pos, is_tracked_box = change['new'][0], change['new'][1], change['new'][2]
        
        if self.tracked_bbox != None and is_tracked_box:
    
            fwd_speed = (depth - TARGET_DISTANCE) / TARGET_DISTANCE
            turn_speed = (x_pos - CENTER_X) / CENTER_X
    
            fwd_speed = clamp(fwd_speed)
            turn_speed = clamp(turn_speed)
    
            left_motor_speed = clamp(fwd_speed + turn_speed)
            right_motor_speed = clamp(fwd_speed - turn_speed)
        
            # print(f'Speeds: left_motor_speed: {left_motor_speed:.2f} | right_motor_speed: {right_motor_speed:.2f}', end='\r')
        
            if (left_motor_speed > 0 and right_motor_speed > 0):
                left_motor_speed *= 4
                right_motor_speed *= 4
            elif (left_motor_speed < 0 and right_motor_speed < 0):
                left_motor_speed *= 1.5
                right_motor_speed *= 1.5
            
            
            robot.frontLeft(left_motor_speed)
            robot.backLeft(left_motor_speed)
            robot.frontRight(right_motor_speed)
            robot.backRight(right_motor_speed)
        else:
            robot.stop()

        
        
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

def clamp(value):
    return max(-1, min(1, value))

camera = Camera()
camera.start()

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


[2025-05-07 11:15:57 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:15:57 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:15:57 UTC][ZED][INFO] Logging level INFO
[2025-05-07 11:15:57 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-05-07 11:15:58 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-05-07 11:15:58 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-05-07 11:15:58 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-05-07 11:15:58 UTC][ZED][INFO] [Init]  Serial Number: S/N 35159485
Loading yolo11l_half.engine for TensorRT inference...
[05/07/2025-12:15:59] [TRT] [I] The logger passed into createInferRuntime differs from one already provided for an existing builder, runtime, or refitter. Uses of the global logger, returned by nvinfer1::getLogger(), will return the existing value.
[05/07/2025-12:15:59] [TRT] [I] Loaded engine size: 52 MiB
[05/07/2025-12:15:59] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely t

### Stop

In [18]:
camera.stop()
robot.stop()